# Display Wav

## Preparation

In [1]:
!pip install pydub

In [2]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import csv
from scipy.io import wavfile
import shutil
from pydub import AudioSegment
from datetime import datetime, timedelta
from moviepy.editor import VideoFileClip
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Function Definition

### Execute function in subfolders

In [3]:
def execute_function_in_subfolders(folder_path, function):
    for root, dirs, files in os.walk(folder_path):
        for dir in dirs:
            subfolder_path = os.path.join(root, dir)
            function(subfolder_path)

### Create text files

In [4]:
def create_txt_file(folder_path, data):
    # Create the file name by combining the folder name with 'txtfile'
    folder_name = os.path.basename(folder_path)
    file_name = folder_name.split('_')[0] + '_eurythmy_detection.txt'

    # Create the full file path
    file_path = os.path.join(folder_path, file_name)

    # Extract the directory path without the file name
    dir_path = os.path.dirname(file_path)

    # Create the directory if it doesn't exist
    os.makedirs(dir_path, exist_ok=True)

    # Write the data to the text file
    with open(file_path, 'w') as file:
        for row in data:
            file.write('\t'.join(str(item) for item in row))
            file.write('\n')

    print(f"Text file '{file_name}' created in the folder '{folder_name}'.")

### Cut the wav file

In [5]:
from datetime import timedelta

def cut_wav_file(file_path, start, end):
    """
    Cuts a WAV file from the specified start time to the specified end time.

    Args:
        file_path (str): The path to the input WAV file.
        start (float): The start time in seconds.
        end (float): The end time in seconds.

    Returns:
        AudioSegment: The cut audio segment.

    Raises:
        FileNotFoundError: If the input file does not exist.
        ValueError: If the start or end time is invalid.
        Exception: If an error occurs during file cutting.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Input file '{file_path}' does not exist.")

    if start < 0 or end < 0 or start >= end:
        raise ValueError("Invalid start or end time.")

    try:
        start_time = timedelta(seconds=start)
        end_time = timedelta(seconds=end)

        audio = AudioSegment.from_file(file_path)
        start_ms = int(start_time.total_seconds() * 1000)
        end_ms = int(end_time.total_seconds() * 1000)
        cut_audio = audio[start_ms:end_ms]

        return cut_audio

    except Exception as e:
        raise Exception("Error occurred during file cutting.") from e



### Cut Mp4 File

In [6]:
def cut_mp4_file(file_path, start, end):
    """
    Cuts an MP4 file from the specified start time to the specified end time.

    Args:
        file_path (str): The path to the input MP4 file.
        start (float): The start time in seconds.
        end (float): The end time in seconds.

    Returns:
        VideoFileClip: The cut video clip.

    Raises:
        FileNotFoundError: If the input file does not exist.
        ValueError: If the start or end time is invalid.
        Exception: If an error occurs during file cutting.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Input file '{file_path}' does not exist.")

    if start < 0 or end < 0 or start >= end:
        raise ValueError("Invalid start or end time.")

    try:
        start_time = start
        end_time = end
        clip = VideoFileClip(file_path).subclip(start_time, end_time)

        return clip

    except Exception as e:
        raise Exception("Error occurred during file cutting.") from e

### Find text file

In [7]:
def find_text_file(file_list):
    # Find and return the first text file in the list
    for file in file_list:
        if file.endswith('.txt'):
            return file
    return None

### Process File

In [8]:
def process_file(file_path,df,directory_out):


  file_name= os.path.basename(file_path)
  file_out = os.path.join(directory_out, file_name)
  print(file_out)

  for index, row in df.iterrows():
        start = row['start_seconds']
        end = row['end_seconds']
        if end-start > 18:
          print('##### MAL: ',file_path,start,end)

        if file_path.endswith('.mp4'):
            cut_file= cut_mp4_file(file_path, start, end)
            output_file_path = file_out[:-11] + index + "_trimmed.mp4"
            cut_file.write_videofile(output_file_path)
        elif file_path.endswith('.wav'):
            cut_file= cut_wav_file(file_path, start, end)
            output_file_path = file_out[:-11] + index + "_trimmed.wav"
            cut_file.export(output_file_path, format="wav")
        print(start,end,output_file_path)


### Time to seonds

In [9]:
def time_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    total_seconds = minutes * 60 + seconds
    return total_seconds

### Process files in folder

In [10]:
def process_files(directory_out, file_list):
    # Find the text file in the list
    text_file = find_text_file(file_list)

    if text_file:
        # Read the contents of the text file
        with open(text_file, 'r') as file:
            #df = pd.read_csv(text_file, header=None, index_col=0, names= ['id', 'start', 'end'] )
            df = pd.read_csv(text_file, index_col=0,header=0, names=['id', 'start', 'end'])
        # Apply time-to-seconds conversion and create new columns
        df['start_seconds'] = df['start'].apply(time_to_seconds)
        df['end_seconds'] = df['end'].apply(time_to_seconds)

        # Iterate through the remaining files
        for file in file_list:
            if file != text_file:
                process_file(file,df,directory_out)


### Process directory

In [11]:
def process_directory(directory_path):
    # Get the directory name
    directory_name = os.path.basename(directory_path)

    # Create a new directory with the modified name
    new_directory_name = directory_name[:-8] + '_trimmed'
    new_directory_path = os.path.join(os.path.dirname(directory_path), new_directory_name)

    # Check if the directory already exists
    if not os.path.exists(new_directory_path):
        os.makedirs(new_directory_path)
        print(f"New directory '{new_directory_name}' created at '{new_directory_path}'.")

        # Get a list of all the files in the original directory
        file_list = []
        for root, _, files in os.walk(directory_path):
            for file in files:
                file_path = os.path.join(root, file)
                file_list.append(file_path)

        # Call another function with the new directory path and file list as parameters
        process_files(new_directory_path, file_list)

    else:
        print(f"Directory '{new_directory_name}' already exists at '{new_directory_path}'.")




## Use of Function

In [12]:
folder_path= r'/content/drive/MyDrive/plants/data/2. reduced/230509 SE_reduced'

In [13]:
#process_directory(folder_path)

In [14]:
def process_subdirectories(root_directory):
    for entry in os.scandir(root_directory):
        if entry.is_dir():
            subdirectory = entry.path
            try:
                print('\n### Processing',subdirectory)
                process_directory(subdirectory)
            except OSError as e:
                print(f"Error processing files in directory: {subdirectory}")
                print(f"Error message: {str(e)}")


# Provide the root directory
root_directory = r'/content/drive/MyDrive/plants/data/2. reduced/new'

# Call the function to process subdirectories
process_subdirectories(root_directory)


### Processing /content/drive/MyDrive/plants/data/2. reduced/new/230809 BE_reduced
New directory '230809 BE_trimmed' created at '/content/drive/MyDrive/plants/data/2. reduced/new/230809 BE_trimmed'.
/content/drive/MyDrive/plants/data/2. reduced/new/230809 BE_trimmed/230809_BE2_M_reduced.mp4
Moviepy - Building video /content/drive/MyDrive/plants/data/2. reduced/new/230809 BE_trimmed/230809_BE2_M_A1_trimmed.mp4.
MoviePy - Writing audio in 230809_BE2_M_A1_trimmedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/plants/data/2. reduced/new/230809 BE_trimmed/230809_BE2_M_A1_trimmed.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/plants/data/2. reduced/new/230809 BE_trimmed/230809_BE2_M_A1_trimmed.mp4
17 30 /content/drive/MyDrive/plants/data/2. reduced/new/230809 BE_trimmed/230809_BE2_M_A1_trimmed.mp4
Moviepy - Building video /content/drive/MyDrive/plants/data/2. reduced/new/230809 BE_trimmed/230809_BE2_M_G1_trimmed.mp4.
MoviePy - Writing audio in 230809_BE2_M_G1_trimmedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/plants/data/2. reduced/new/230809 BE_trimmed/230809_BE2_M_G1_trimmed.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/plants/data/2. reduced/new/230809 BE_trimmed/230809_BE2_M_G1_trimmed.mp4
33 45 /content/drive/MyDrive/plants/data/2. reduced/new/230809 BE_trimmed/230809_BE2_M_G1_trimmed.mp4
Moviepy - Building video /content/drive/MyDrive/plants/data/2. reduced/new/230809 BE_trimmed/230809_BE2_M_D1_trimmed.mp4.
MoviePy - Writing audio in 230809_BE2_M_D1_trimmedTEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_L2_trimmed.mp4
213 222 /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_L2_trimmed.mp4
Moviepy - Building video /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_L3_trimmed.mp4.
MoviePy - Writing audio in 230807_BJ1_M_L3_trimmedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_L3_trimmed.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_L3_trimmed.mp4
223 232 /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_L3_trimmed.mp4
Moviepy - Building video /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_L4_trimmed.mp4.
MoviePy - Writing audio in 230807_BJ1_M_L4_trimmedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_L4_trimmed.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_L4_trimmed.mp4
234 243 /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_L4_trimmed.mp4
/content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_BYB_Recording_2023-08-07_09.29.36_reduced.wav
21 35 /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_BYB_Recording_2023-08-07_09.29.36_A1_trimmed.wav
38 50 /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_BYB_Recording_2023-08-07_09.29.36_G1_trimmed.wav
52 64 /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_BYB_Recording_2023-08-07_09.29.36_D1_trimmed.wav
67 78 /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_BJ1_M_BYB_Recording_2023-08-07_09.29.36_A2_trimmed.wav
80 91 /content/drive/MyDrive/plants/data/2. reduced/new/230807 BJ1_trimmed/230807_B